Trying to figure out why the process of making high res maps didn't work for a few of my samples. 

In [2]:
import numpy as np
import astropy.units as u
from astropy.cosmology import z_at_value

from lenstools.simulations.fastpm import FastPMSnapshot
from lenstools.simulations.raytracing import DensityPlane, PotentialPlane, RayTracer
from lenstools.image.convergence import ConvergenceMap
import bigfile

from os import path
from glob import glob
from shutil import copy

In [57]:
class MyFastPMSnapshot(FastPMSnapshot):
    def getHeader(self):

        #fastpm doesn't always save thigns in teh way that lenstools wants
        basename = path.realpath(self.fp.basename)
        header_files = glob(basename[:-2] + '/Header/*')
        for hf in header_files:
            copy(hf, basename)
        # Initialize header
        header = dict()
        bf_header = self.fp["."].attrs
        ###############################################
        # Translate fastPM header into lenstools header#
        ###############################################

        # Number of particles/files
        header["num_particles_file"] = bf_header["NC"][0] ** 3
        header["num_particles_total"] = header["num_particles_file"]
        header["num_files"] = 1

        # Cosmology
        header["Om0"] = bf_header["OmegaM"][0]
        header["Ode0"] = 1. - header["Om0"]
        header["w0"] = -1.
        header["wa"] = 0.
        header["h"] = bf_header['HubbleParam'][0]

        # Box size in kpc/h
        header["box_size"] = bf_header["BoxSize"][0] * 1.0e3

        # Masses
        header["masses"] = bf_header['MassTable'] * header["h"]
        print(header)
        #################

        return header
    def getPositions(self, first=None, last=None, save=True):

        # Get data pointer
        data = bigfile.BigData(self.fp)
        print(data.shape)
        assert False
        # Read in positions in Mpc/h
        if (first is None) or (last is None):
            positions = data["Position"][:] * self.Mpc_over_h
        # aemit = data["Aemit"][:]
        else:
            positions = data["Position"][first:last] * self.Mpc_over_h
        # aemit = data["Aemit"][first:last]
        print(len(positions))
        # Enforce periodic boundary conditions
        for n in (0, 1):
            positions[:, n][positions[:, n] < 0] += self.header["box_size"]
            positions[:, n][positions[:, n] > self.header["box_size"]] -= self.header["box_size"]

        # Maybe save
        if save:
            self.positions = positions
        # self.aemit = aemit

        # Initialize useless attributes to None
        self.weights = None
        self.virial_radius = None
        self.concentration = None
        # Return
        print(len(positions))
        print(positions)
        return positions


In [58]:
def conv_in_fov(lens_planes, z_lens, start_coord, fov = 10*u.deg, fov_resolution = 256):

    coords = np.linspace(0, fov.value, fov_resolution)
    mesh_coords = np.meshgrid(coords, coords)

    pos = (np.array(mesh_coords) + np.array(start_coord).reshape((-1, 1,1)) ) * fov.unit
    #print pos.shape
    conv_born = lens_planes.convergenceBorn(pos, z=z_lens, save_intermediate=False)
    #print conv_born.data.shape
    conv_born = ConvergenceMap(conv_born, angle=fov)

    return conv_born.data


In [59]:
def compute_potential_planes(snap, z_source, num_lenses, res=8192, smooth=1):
    chi_max = snap.cosmology.comoving_distance(z_source)

    thickness = chi_max / num_lenses
    chi_start = thickness / 2
    chi_end = chi_max - thickness / 2

    # Lens centers
    chi_centers = np.linspace(chi_start.value, chi_end.value, num_lenses) * chi_max.unit
    tracer = RayTracer()
    for i,chi in enumerate(chi_centers):
        zlens = z_at_value(snap.cosmology.comoving_distance,chi)
        #print chi, thickness, res, smooth
        d,r,n = snap.cutPlaneGaussianGrid(normal=2,center=chi,thickness=thickness,plane_resolution=res,kind="potential", smooth=smooth)

        lens = PotentialPlane(d.value,angle=snap.header["box_size"],comoving_distance=chi,redshift=zlens,cosmology=snap.cosmology,unit=u.rad**2)
        tracer.addLens(lens)

    #Add a fudge lens at the end (needed by ODE solver implementation)
    chi_fudge = chi_end + thickness
    z_fudge = 1000.
    tracer.addLens(PotentialPlane(np.zeros((res,res)),angle=snap.header["box_size"],redshift=z_fudge,comoving_distance=chi_fudge,cosmology=snap.cosmology,num_particles=None))
    #Order lenses
    tracer.reorderLenses()
    return tracer


In [60]:
def convert_particles_to_convergence(lightcone_dir, ang_size_image=4, 
                                     ang_space_patches =None,
                                     fov_resolution = 512,
                                     potential_kwargs ={} ):

    snap = MyFastPMSnapshot.open(lightcone_dir)
    potential_defaults = {'z_source': 0.3, 'num_lenses': 25}
    potential_defaults.update(potential_kwargs)
    #print 'snap',len(snap.positions), snap.positions
    lens_planes = compute_potential_planes(snap, **potential_defaults)
    z = potential_defaults['z_source']
    if ang_space_patches is None:
        ang_space_patches = ang_size_image

    n_sub = int( (360.0-ang_size_image)/ang_space_patches +1 )

    conv = np.zeros((int(n_sub**2), fov_resolution, fov_resolution))
    for i, ra in enumerate(np.arange(0.0, 360.0, ang_space_patches)):
        for j,dec in enumerate(np.arange(0.0, 360.0, ang_space_patches)):
            #print i,j, ra, dec
            conv[i*n_sub+j] = conv_in_fov(lens_planes, z, (ra, dec), fov=ang_size_image * u.deg, fov_resolution=fov_resolution)

    boxno = int(lightcone_dir[-16:-13])
    np.save(path.join(lightcone_dir, 'proj_map_%d_%03d.npy'%(fov_resolution, boxno)), conv)



In [63]:
boxno = 11
particle_path = '/home/users/swmclau2/scratch/UatuFastPM/Box%03d/lightcone/1/'%boxno

In [64]:
convert_particles_to_convergence(particle_path)

{'num_particles_file': 134217728, 'num_particles_total': 134217728, 'num_files': 1, 'Om0': 0.284745, 'Ode0': 0.715255, 'w0': -1.0, 'wa': 0.0, 'h': 0.7, 'box_size': 1024000.0, 'masses': array([ 0.        , 44.24219743,  0.        ,  0.        ,  0.        ,
        0.        ])}
(134206871,)


AssertionError: 